# A Collection of Results from OpenWPM data

To use, enter in the `datadir_path` bellow, then run all cells

In [ ]:
from pathlib import Path
datadir_path : Path = Path('')

In [ ]:
import sys
import os
from sqlalchemy.engine import Engine
from sqlalchemy import create_engine, URL 
import plyvel
from typing import Any
%reload_ext sql

parent_path : Path = Path(sys.path[0]).parent.resolve()
if str(parent_path) not in sys.path:
    sys.path.append(str(parent_path))


database_url : URL = URL.create(drivername = "sqlite", database = str(datadir_path.joinpath("crawl-data.sqlite")) )
os.environ["DATABASE_URL"] = str(database_url)
engine : Engine = create_engine(database_url)
if 'db' not in locals():
    db : Any = plyvel.DB( str(datadir_path.joinpath("leveldb")) ) #type: ignore

### Prevalence of `CanvasRenderingContext2D` versus `WebGLRenderingContext`

In [ ]:
%%sql
WITH a AS (
    SELECT COUNT(DISTINCT visit_id) AS Total
    FROM http_responses WHERE 
    response_status LIKE "2%"
), b AS (
    SELECT COUNT(DISTINCT visit_id) AS Using_Canvas
    FROM javascript
    WHERE symbol LIKE "CanvasRenderingContext2D%"
), c AS (
    SELECT COUNT(DISTINCT visit_id) AS Using_WEBGL
    FROM javascript
    WHERE symbol LIKE "WebGLRenderingContext%"
)
SELECT a.Total, b.Using_Canvas, c.Using_WEBGL
FROM a, b, c